In [2]:
import torch
from torch import nn
from torch.nn import functional as F


In [ ]:
kl_loss = nn.KLDivLoss(reduction = "batchmean")
#平均KL散度

batchmean 按batch大小取平均

先算出每个样本的KL散度（把样本所有类别的差异加起来）
除以Batch Size

In [4]:
input = torch.log(torch.FloatTensor([[1/3,1/3,1/3]]))
print(input.shape,input)
target = torch.FloatTensor([[9/25,12/25,4/25]])
print(target.shape)

torch.Size([1, 3]) tensor([[-1.0986, -1.0986, -1.0986]])
torch.Size([1, 3])


In [5]:
kl_loss(input,target)

tensor(0.0853)

DKL(P||Q) = sigma(PlnP - PlnQ)

L(y_pred , y_true) = y_true * (ln(y_true) - ln(y_pred))


In [6]:
kl_loss = nn.KLDivLoss(reduction = "batchmean")
ce_loss = nn.CrossEntropyLoss()

In [7]:
input = torch.randn(3,5,requires_grad=True)
input

tensor([[-0.1334,  1.7769, -1.2352, -0.4165,  1.1723],
        [-1.3531,  2.6970, -0.3182,  0.8409,  0.0530],
        [ 0.5969,  0.0208,  1.1534,  0.1029,  0.7332]], requires_grad=True)

In [9]:
target = torch.empty(3,dtype=torch.long).random_(5)
target

tensor([2, 1, 1])

随机生成3个正确答案
类型只能是long
random_  "_" 原地修改，不需要生成新变量
5 生成的范围是 [0,5)

In [10]:
ce_value = ce_loss(input,target)
ce_value

tensor(2.0285, grad_fn=<NllLossBackward0>)

In [11]:
input_log_softmax = F.log_softmax(input,dim=1)
input_log_softmax

tensor([[-2.5283, -0.6179, -3.6301, -2.8113, -1.2225],
        [-4.3078, -0.2576, -3.2728, -2.1137, -2.9016],
        [-1.6219, -2.1980, -1.0654, -2.1159, -1.4856]],
       grad_fn=<LogSoftmaxBackward0>)

log_softmax:
先做softmax归一化 变成概率
再做log 取自然对数

dim=1
input现在是3*5的表格 （3 个 样本，5个类别）
dim = 1
沿着列的方向（横向）进行计算 （一行 的5个数做softmax）


In [14]:
target_shaped = torch.FloatTensor([[0,0,1,0,0],
                                   [0,1,0,0,0],
                                   [0,1,0,0,0]])

In [15]:
kl_loss(input_log_softmax , target_shaped)

tensor(2.0285, grad_fn=<DivBackward0>)

kl_loss的接口
对于input 需要是个prob distribution，还需要取log
所以要用到
log_softmax

In [16]:
ce_loss(input,target_shaped)

tensor(2.0285, grad_fn=<DivBackward1>)

注意到此时ce_loss 和 kl_loss一致
即 H(p,q) = H(p,q) - H(p)
故 可知 H(p) = 0
独热编码 有H(p) =  0 

In [ ]:
ce_loss(target_shaped,target_shaped)

TypeError: softmax() received an invalid combination of arguments - got (Tensor), but expected one of:
 * (Tensor input, int dim, torch.dtype dtype = None, *, Tensor out = None)
 * (Tensor input, name dim, *, torch.dtype dtype = None)


这样计算的不是独热变量target_shaped的H
ce_loss的第一个参数是分数logits，而不是概率,传进来会先做一次softmax

In [24]:
from torch.distributions import Categorical

# 告诉它 target_shaped 是概率(probs)
entropy = Categorical(probs=target_shaped).entropy()

print(entropy) 
# 输出: tensor([0., 0., 0.]) 
# 解释: 结果全是 0，因为确定性事件（独热编码）的熵就是 0。

tensor([1.1921e-07, 1.1921e-07, 1.1921e-07])


In [25]:

# 给 log 里面加一个极小数 1e-9 防止报错，因为 log(0) 无定义
H = -torch.sum(target_shaped * torch.log(target_shaped + 1e-9), dim=1)

print(H)
# 输出: tensor([0., 0., 0.])

tensor([-0., -0., -0.])


In [18]:
kl_loss = torch.nn.KLDivLoss(reduction='none')
ce_loss = torch.nn.CrossEntropyLoss(reduction = 'none')


In [19]:
#q(x)
input = torch.Tensor([[-0.1,0.2,-0.4,0.3]])
#p(x)
target = torch.Tensor([[-0.7,0.1,-0.1,0.1]])


In [ ]:
# kl(p,q)
kl_output = kl_loss(F.log_softmax(input,dim=1),torch.softmax(target,dim=1))
print(kl_output)
print(kl_output.mean())
print(kl_output.sum())

tensor([[-0.0635,  0.0116,  0.1097, -0.0190]])
tensor(0.0097)
tensor(0.0389)


In [27]:
#H(p,q)
a = ce_loss(input , torch.softmax(target , dim=1))
print(a)

tensor([1.3832])


In [28]:
#H(p) = H(p , p)
b = ce_loss(target , torch.softmax(target , dim=1))
print(b)

tensor([1.3443])


In [29]:
a - b

tensor([0.0389])

和 kl_output.sum()一致